In [86]:
import dum_tv.apps.model.main as main
from pathlib import Path
import dum_tv as pkg

configs_path =main.configs_path
__file__ = Path(Path(pkg.__file__).parent,"apps/model/main.py")
print(__file__)

/home/tjrym/workspace/DUM-TV/dum_tv/apps/model/main.py


In [87]:
from datetime import datetime
import copy
d = {
        "experiments":{}
}

dexp = d["experiments"]

dd = {
        "note":"Toy denoise",
        "datetime":f"{datetime.now()}",
        "compiler":"compile_iteration_tv",
        "config":{
                "patch_shape":(3,256,256), 
                "n_iteration":40, # depth of DU
                "varia_d": {
                    "kO":2,
                    "kR":1,
                    "C":3,
                    "kerK":"DxDy",
                    "beta":0.04,
                    "rho":0.8
                },
                "varia_d-name":"Toy",
        }
}

dexp["iteration"] = dd



In [88]:

dd = {
        "note":"Toy denoise",
        "datetime":f"{datetime.now()}",
        "compiler":"compile_training_tv",
        "config":{
                "patch_shape":(3,256,256), 
                "n_iteration":64, # depth of DU
                "varia_d": {
                    "kO":4,
                    "kR":5,
                    "C":3,
                    "kerK":"DxDy",
                    "beta":0.04,
                    "rho":0.8
                },
                "varia_d-name":"Toy",
            "lr":2e-4,
            "lr_scheduler_frequency":2000,
            "max_epochs":20,
        },
        
}
dexp['training'] = dd

- varia_d --> Varia:dataclasses
- Varia --> controled_init: list-(str,tensor); shape
- ctrlnet_d, controled_init --> CtrlNet:Module
- dum_d, controled_init, CtrlNet --> TVNet:Module
- model_d,... --> Model, Trainer, Runner


In [1]:
C,H,W = 3,256,256
dd = {
    "note": "With CtrlNet",
    "datetime":f"{datetime.now()}",
    "compiler":"compile_training_tv_ctrl",
    "config":{
        # ==== variables in variational model and solver
        "varia_d":{
            "kO":4,
            "kR":5,
            "C":3,
            "kerK":"DxDy",
            "beta":0.04,
            "rho":0.8
        }, # Varia(**varia_d)
        "varia_d-name":"", # comment
        
        # ==== control-net
        "ctrlnet_d":{
            "branches_d":{
                (C,H,W),(1,)
            },
        }, # vars(.)[ctrlnet_d_class](**ctrlnet_d)
        "ctrlnet_d_class":"Alexnet_Branch" ,# class in apps/model/ctrlnet,
        "ctrlnet_d-name":"Alex", # comment
        
        # ==== deep unfolding network
        "dum_d":{
            "patch_shape":(C,H,W), 
            "n_iteration":64, # depth of DU
            "param_group":["model","solver","last-2-layers","all"]
        },
        
        # ==== model and trainer
        "model_d":{
            "lr":2e-4,
            "lr_scheduler_frequency":2000, # steps of descend
            "max_epochs":20,
            "scheduler": # trigger by i_epoch or monitered maetrics
                {"model_param_only":[],
                 "solver_param_only":[],
                 "trancte_backprop_to_last_layer":[4],
                 "alternative_training":["solver","model","solver"]
                 }
                
        }
    },
}
dexp[""]

SyntaxError: invalid syntax (2197831341.py, line 25)

## Dump to disk

In [89]:

from ren_utils.rennet import call_by_inspect,getitems_as_dict,RenNetDumper,RenNetLoader,add_representers, add_constructors

import yaml

# def rep_varia_ins(dumper, data:Variable_Instructions, tag):
#         return dumper.represent_mapping(tag,dict(vars(data)))
# 
# def con_varia_ins(loader, node, tag):
#         d= loader.construct_mapping(node, deep=True)
#         v=  Variable_Instructions(**d)
#         return v
# 
# add_representers(RenNetDumper,[rep_varia_ins],prefix="rep")
# add_constructors(RenNetLoader,[con_varia_ins],prefix="con")

from pathlib import Path
import dum_tv
root_pkg = Path(dum_tv.__file__).parent
assert __file__.exists(), str(__file__)
_config_yaml = ".".join(Path(__file__).relative_to(root_pkg).parts)
_config_yaml = Path(root_pkg,"../Scripts",_config_yaml)
_config_yaml = Path(_config_yaml.parent,_config_yaml.stem+".yaml")

import os
if _config_yaml.exists():
        os.remove(_config_yaml)

with open(_config_yaml,"x") as f:
        yaml.dump(d,f,Dumper=RenNetDumper)



In [90]:
import yaml
from ren_utils.rennet import call_by_inspect,getitems_as_dict,RenNetDumper,RenNetLoader

with open(_config_yaml,"r") as f:
        d_ = yaml.load(f,Loader=RenNetLoader)


from pprint import pprint
pprint(list(d_["experiments"].keys()))

['iteration', 'training']


In [91]:

pprint(d_)

{'experiments': {'iteration': {'compiler': 'compile_iteration_tv',
                               'config': {'n_iteration': 40,
                                          'patch_shape': [3, 256, 256],
                                          'varia_d': {'C': 3,
                                                      'beta': 0.04,
                                                      'kO': 2,
                                                      'kR': 1,
                                                      'kerK': 'DxDy',
                                                      'rho': 0.8},
                                          'varia_d-name': 'Toy'},
                               'datetime': '2023-09-15 00:05:45.939378',
                               'note': 'Toy denoise'},
                 'training': {'compiler': 'compile_training_tv',
                              'config': {'lr': 0.0002,
                                         'lr_scheduler_frequency': 2000,
                    